In [1]:
import xarray as xr
from glob import glob

In [2]:
all_files = sorted(glob('/glade/work/dkimpara/CREDIT_runs/prescribed_skebs_rollout//std_2012-06-05T00Z/*'))


In [3]:
datasets = []
for file in all_files[:100]:
    datasets.append(xr.open_dataset(file))

DS = xr.concat(datasets, dim='time')

In [4]:
DS

<xarray.Dataset> Size: 641MB
Dimensions:    (time: 20, level: 32, latitude: 192, longitude: 288)
Coordinates:
  * time       (time) datetime64[ns] 160B 2012-06-05T06:00:00 ... 2012-06-10
  * level      (level) int64 256B 0 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * latitude   (latitude) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * longitude  (longitude) float32 1kB 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
Data variables: (12/21)
    U          (time, level, latitude, longitude) float32 142MB 0.08721 ... 2...
    V          (time, level, latitude, longitude) float32 142MB 0.2616 ... 2.52
    T          (time, level, latitude, longitude) float32 142MB 0.0 ... 1.266
    Qtot       (time, level, latitude, longitude) float32 142MB 0.0 ... 0.000...
    PS         (time, latitude, longitude) float32 4MB 0.0 0.0 ... 206.5 194.4
    TREFHT     (time, latitude, longitude) float32 4MB 0.0 0.0 ... 1.256 1.266
    ...         ...
    FSNS       (time, latitude, longitude) float32 4MB 0.0 0.0 ... 1.679e+05
    FLNS       (time, latitude, longitude) float32 4MB 0.0 0.0 ... 1.449e+05
    FSNT       (time, latitude, longitude) float32 4MB 0.0 0.0 ... 1.525e+05
    FLNT       (time, latitude, longitude) float32 4MB 0.0 0.0 ... 5.067e+04
    SHFLX      (time, latitude, longitude) float32 4MB 0.0 0.0 ... 4.166e+04
    LHFLX      (time, latitude, longitude) float32 4MB 0.0 0.0 ... 3.751e+04
Attributes:
    Conventions:  CF-1.11

In [6]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import cartopy.crs as ccrs
import numpy as np
import cmocean
from tqdm import tqdm
import multiprocessing as mp
import os

# Parameters for customization
central_latitude = 23.5  # Earth's axial tilt ~23.5°
fps = 12
output_file = "Robinson_U10_dynamic_earth.mp4"

# Assuming `DS` is a dataset object with variables 'longitude', 'latitude', 'PRECT', and 'time'
print('starting')

# Data grid
lons = DS['longitude'].values
lats = DS['latitude'].values
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Wrap longitude grid for smooth transitions
lon_grid_wrapped = np.concatenate((lon_grid, lon_grid[:, :1] + 360), axis=1)
lat_grid_wrapped = np.concatenate((lat_grid, lat_grid[:, :1]), axis=1)  # Adjust lat_grid for consistency
data_np = DS['U10'].values

# Wrap data for smooth transitions
data_wrapped = np.concatenate((data_np, data_np[:, :, :1]), axis=2)

# Function to generate a single frame
def generate_frame(frame):
    fig = plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=180))
    ax.set_global()
    ax.coastlines()

    vma = 20
    data = data_wrapped[frame, :, :]

    # Plot the data
    img = ax.contourf(
        lon_grid_wrapped, lat_grid_wrapped, data, 60, transform=ccrs.PlateCarree(), cmap=cmocean.cm.curl, vmin=0, vmax=vma
    )

    # Add a title based on time
    current_time = str(DS['time'].values[frame])[:10]  # Truncate to the day
    ax.set_title(f"Time: {current_time}", fontsize=14)

    # Save the frame
    frame_file = f"frame_{frame:06d}.png"
    plt.savefig(frame_file, dpi=100, bbox_inches='tight')
    plt.close(fig)
    return frame_file

# Parallel frame generation in batches
frames = np.arange(0, 100, 1)
batch_size = 50
frame_files = []

for batch_start in range(0, len(frames), batch_size):
    batch_frames = frames[batch_start:batch_start + batch_size]
    with tqdm(total=len(batch_frames), desc=f"Rendering batch {batch_start // batch_size + 1}") as pbar:
        with mp.Pool(processes=32) as pool:
            for frame_file in pool.imap_unordered(generate_frame, batch_frames):
                frame_files.append(frame_file)
                pbar.update(1)

# Debugging: Check if all frames are generated
for frame_file in frame_files:
    if not os.path.exists(frame_file):
        print(f"Warning: Frame file {frame_file} does not exist.")

# Combine frames into video using ffmpeg
print("Combining frames into video...")
if frame_files:
    # os.system("ls frame_*.png")  # List generated files for debugging
    os.system(f"ffmpeg -r {fps} -pattern_type glob -i 'frame_*.png' -vf 'scale=trunc(iw/2)*2:trunc(ih/2)*2' -vcodec libx264 -pix_fmt yuv420p -y {output_file}")
else:
    print("No frames found to combine.")

print("Animation saved as", output_file)

starting


Rendering batch 1: 100%|██████████| 20/20 [00:23<00:00,  1.16s/it]
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfr

Combining frames into video...
Animation saved as Robinson_U10_dynamic_earth.mp4


Output #0, mp4, to 'Robinson_U10_dynamic_earth.mp4':
  Metadata:
    encoder         : Lavf61.7.100
  Stream #0:0: Video: h264 (avc1 / 0x31637661), yuv420p(tv, progressive), 778x428 [SAR 779:778 DAR 779:428], q=2-31, 12 fps, 12288 tbn
      Metadata:
        encoder         : Lavc61.19.100 libx264
      Side data:
        cpb: bitrate max/min/avg: 0/0/0 buffer size: 0 vbv_delay: N/A
[out#0/mp4 @ 0x55ab9eec2b80] video:186KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.582770%
frame=   20 fps=0.0 q=-1.0 Lsize=     187KiB time=00:00:01.50 bitrate=1023.6kbits/s speed=2.51x    
[libx264 @ 0x55ab9eec43c0] frame I:1     Avg QP:27.02  size: 36778
[libx264 @ 0x55ab9eec43c0] frame P:6     Avg QP:23.60  size: 14418
[libx264 @ 0x55ab9eec43c0] frame B:13    Avg QP:23.49  size:  5141
[libx264 @ 0x55ab9eec43c0] consecutive B-frames: 10.0% 10.0%  0.0% 80.0%
[libx264 @ 0x55ab9eec43c0] mb I  I16..4:  3.6% 55.3% 41.1%
[libx264 @ 0x55ab9eec43c0] mb P  I16..4:  2.3% 2